In [1]:
# Add the parent directory of the current working directory to the Python path at runtime. 
# In order to import modules from the src directory.
import os
import sys 

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from src.stat_utils import *
from src.anl_utils import load_data, get_session_data

In [3]:
sim_results_folder = '../results/simulation'
data_folder = '../data'

sim_at_file = os.path.join(sim_results_folder, 'highres_arnold_tongues.npy')
emp_at_file = os.path.join(data_folder, 'Experiment.csv')

### Use model as predictor of empirical results

In [4]:
# Load the simulations results and the empirical data
sim_results = np.load(sim_at_file)[0][:,::6, ::6]
sim_results_vector = sim_results.mean(axis=0).flatten()

data = load_data(emp_at_file)
# Get the data for the first session
data = get_session_data(data, 1)

# Map synchrony values to each condition in your DataFrame
data['Synchrony'] = data['Condition'].apply(lambda x: sim_results_vector[x-1])

In [5]:
# Define distribution and covariance structure for GEE
family = sm.families.Binomial()
covariance_structure = sm.cov_struct.Exchangeable()

# Fit the GEE model
model = smf.gee("Correct ~ Synchrony",
                    "SubjectID",
                    data,
                    cov_struct=covariance_structure,
                    family=family)
results = model.fit()

print(results.summary())

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:         Exchangeable   Num. iterations:                     6
Date:                     Wed, 17 Jul 2024   Scale:                           1.000
Covariance type:                    robust   Time:                         13:38:24
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1115      0.037      3.002      0.003       0.039       0.184
Synchro

In [6]:
print_wald_chi_square(results)

Wald Chi-Square:
+-----------+--------------------+-----------------------+
|  Variable |     Chi-Square     |        p-value        |
+-----------+--------------------+-----------------------+
| Intercept | 9.012156884475772  |  0.002681897414354512 |
| Synchrony | 20.923917102628227 | 4.778904663696652e-06 |
+-----------+--------------------+-----------------------+


### Analyze model in the same way as empirical results

In [7]:
# Create predictor variables
min_contrast_heterogeneity = data['ContrastHeterogeneity'].min()
max_contrast_heterogeneity = data['ContrastHeterogeneity'].max()
min_grid_coarseness = data['GridCoarseness'].min()
max_grid_coarseness = data['GridCoarseness'].max()

# Create meshgrid for predictors
num_blocks, num_grid_coarseness, num_contrast_heterogeneity = sim_results.shape
grid_coarseness, contrast_heterogeneity = np.meshgrid(
    np.linspace(min_grid_coarseness, max_grid_coarseness, num_grid_coarseness),
    np.linspace(min_contrast_heterogeneity, max_contrast_heterogeneity, num_contrast_heterogeneity)
)

# Expand predictors to match sim_results shape
grid_coarseness_expanded = np.broadcast_to(grid_coarseness, sim_results.shape)
contrast_heterogeneity_expanded = np.broadcast_to(contrast_heterogeneity, sim_results.shape)

# Create DataFrame
df = pd.DataFrame({
    'Synchrony': sim_results.ravel(),
    'ContrastHeterogeneity': contrast_heterogeneity_expanded.ravel(),
    'GridCoarseness': grid_coarseness_expanded.ravel()
})

# Fit logistic regression model
model = smf.logit("Synchrony ~ ContrastHeterogeneity + GridCoarseness + ContrastHeterogeneity*GridCoarseness", data=df)
results = model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.282186
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:              Synchrony   No. Observations:                  750
Model:                          Logit   Df Residuals:                      746
Method:                           MLE   Df Model:                            3
Date:                Wed, 17 Jul 2024   Pseudo R-squ.:                  0.4367
Time:                        13:38:24   Log-Likelihood:                -211.64
converged:                       True   LL-Null:                       -375.72
Covariance Type:            nonrobust   LLR p-value:                 7.945e-71
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               17.0840      1.8